In [1]:
import nutils 
from models.mlp_cat_v3 import *
import common as cm
import torch as th
import SharedArray as sa
import numpy as np

/home/ywang/miniconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
path_dict = {
    0: "/home/ywang/workspace/multask_t0/src/checkpoints/2406120938/0/model_valid/ic_ret=0.274.ckpt",
    1: "/home/ywang/workspace/multask_t0/src/checkpoints/2406120938/1/model_valid/ic_ret=0.248.ckpt",
    2: "/home/ywang/workspace/multask_t0/src/checkpoints/2406120938/2/model_valid/ic_ret=0.246.ckpt",
    3: "/home/ywang/workspace/multask_t0/src/checkpoints/2406111732/3/model_valid/ic_ret=0.230.ckpt",
    4: "/home/ywang/workspace/multask_t0/src/checkpoints/2406111735/4/model_valid/ic_ret=0.226.ckpt",
    5: "/home/ywang/workspace/multask_t0/src/checkpoints/2406111735/5/model_valid/ic_ret=0.223.ckpt",
}

In [ ]:
label_names = ["ret", "mean", "var", "vol", "min", "max", "gap", "nret"]


def get_label(code, cur=0, fut=60, label_idx=[]):

    tag = f"{cur}-{fut}"
    raw_ret = sa.attach(f"label_{code}")
    n = len(raw_ret)

    path = f"/mnt/disk1/multiobj_dataset/{code}"
    selected_labels = [label_names[i] for i in label_idx]
    labels = []
    for label in selected_labels:
        if "ret" not in label:
            name = f"{label}_{tag}.npy"
            labels.append(np.load(f"{path}/{name}").astype(np.float32)[:n])
        elif "nret" in label:
            name = f"ret_{fut}.npy"
            labels.append(np.load(f"{path}/{name}").astype(np.float32)[:n])
    res = np.concatenate([raw_ret] + labels, axis=1)
    return res


def get_data(code):
    x = sa.attach(f"factor_{code}")
    y = sa.attach(f"label_{code}")
    z = sa.attach(f"timestamp_{code}")
    return x, y, z

In [ ]:
codes = cm.SELECTED_CODES
train_end_dates = cm.dates
x_codes, y_codes, ts_codes = [], [], []
for code in codes:
    x, _, ts = get_data(code)
    label = get_label(code, cur=120, fut=180, label_idx=[0, 1, 3, 4, 5])
    x_codes.append(x)
    y_codes.append(label)
    ts_codes.append(ts)

In [ ]:
device = th.device("cuda:1")
pred_y = {i: [] for i in range(6)}

for fold in range(6):
    print(f"processing fold {fold}")
    model = CatNet.load_from_checkpoint(path_dict[fold]).to(device)
    for code in codes:
        test_x = x_codes[codes.index(code)]
        test_ts = ts_codes[codes.index(code)]
        test_x = test_x[
            (test_ts >= train_end_dates[fold]) & (test_ts < train_end_dates[fold + 1])
        ]
        test_x = th.from_numpy(test_x).to(device)
        y_hat = model(test_x)[0].detach().cpu().numpy()
        pred_y[fold].append(y_hat)
        

In [ ]:
pred_ys = []
for i in range(100):
    code = codes[i]
    cur_y = []
    for fold in range(6):
        cur_y.append(pred_y[fold][i])
    cur_y = np.concatenate(cur_y, axis=0)

    ts = ts_codes[i]
    padding_len = sum(ts < trai n_end_dates[0])
    padding_y = np.zeros((padding_len, cur_y.shape[1]))
    cur_y = np.concatenate([padding_y, cur_y], axis=0)
    pred_ys.append(cur_y)
    np.save(f"/mnt/nas/data/WY/factor_0612/{code}.npy", cur_y)